In [2]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [6]:
def distance_cosine(a,b):
    return 1-distance.cosine(a,b)

In [13]:
def distance_euclidean(a,b):
    return distance.euclidean(a,b)

In [38]:
def nearest_neighbor_user(user, topN, simFunc):
    u1 = UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = {}
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = []
        interSectionU2 = []
        if uid == user: continue
            
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim): nn[uid] = sim
            
    return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]
            

In [2]:
def predict_rating(userid, nn=100, simFunc=distance_euclidean):
    neighbor = nearest_neighbor_user(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_movie = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh = 4)
    neighbor_dict = (dict(neighbor))
    ret = []
    
    for movieId, row in neighbor_movie.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0], 0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([movieId, wsum/jsum])
        
    return ret

NameError: name 'distance_euclidean' is not defined

In [40]:
ratings = pd.read_csv('./data/ml-20m/ratings.csv')
ratings.head(5)

,userId,movieId,rating,timestamp
999995,6743,1580,4.0,1117760757
999996,6743,1584,3.0,1117760943
999997,6743,1586,3.0,1117761831
999998,6743,1589,4.0,1117761990
999999,6743,1590,4.0,1117761913


In [42]:
UM_matrix_ds = ratings.pivot(index='userId', columns='movieId', values = 'rating')
UM_matrix_ds.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
selected_user = pd.DataFrame(columns = ratings.columns)
for i in range(0, 10000, 1000):
    selected_user = selected_user.append(ratings.iloc[i])
selected_user.head(10)

,userId,movieId,rating,timestamp
0,1.0,2.0,3.5,1.112486e+09
1000,11.0,527.0,4.5,1.251171e+09
2000,18.0,4967.0,5.0,1.196423e+09
3000,24.0,4321.0,3.0,9.942328e+08
4000,35.0,110.0,4.5,1.164499e+09
5000,50.0,1094.0,4.0,1.182678e+09
6000,54.0,3198.0,2.0,9.754406e+08
7000,59.0,1.0,4.5,1.380401e+09
8000,70.0,2916.0,2.0,1.020294e+09
9000,83.0,342.0,3.5,1.112724e+09


In [44]:
for user in selected_user['userId']:
    print('User {0} neighbors : {1}'.format(user, nearest_neighbor_user(int(user), 3, distance_cosine)))

User 1.0 neighbors : [(6564, 1.0), (6310, 1.0), (5992, 1.0)]
User 11.0 neighbors : [(1801, 1.0), (992, 1.0), (743, 1.0)]
User 18.0 neighbors : [(6098, 1.0), (4929, 1.0), (2615, 1.0)]
User 24.0 neighbors : [(6036, 0.9994838709333602), (6441, 0.9992360733058688), (2725, 0.9989922980237592)]
User 35.0 neighbors : [(6185, 1.0), (5432, 1.0), (4753, 1.0)]
User 50.0 neighbors : [(6541, 1.0), (5641, 1.0), (5308, 1.0)]
User 54.0 neighbors : [(5109, 1.0), (3156, 1.0), (876, 1.0)]
User 59.0 neighbors : [(5733, 1.0), (4992, 1.0), (4416, 1.0)]
User 70.0 neighbors : [(6036, 1.0), (4262, 1.0), (4122, 1.0)]
User 83.0 neighbors : [(6033, 1.0), (5992, 1.0), (5909, 1.0)]


In [46]:
result = []
for i in range(10):
    userId = int(selected_user.iloc[i].userId)
    movieId = int(selected_user.iloc[i].movieId)
    predict = predict_rating(userId, 300, distance_cosine)
    print(predict)
    for movie in predict:
        if movieId == movie[0]:
            result.append([int(userId), int(movieId), movie[1]])
            
resultdf = pd.DataFrame(result, columns=['userId', 'movieId', 'rating'])
resultdf

[[1, 4.078752265273778], [2, 3.9707088748284005], [3, 3.453496819049876], [5, 3.3549224992576985], [6, 3.8513722579241225], [7, 3.3078911478784527], [9, 3.2146241363610533], [10, 3.5107832047986203], [11, 4.035767454192992], [12, 2.375451947006964], [14, 3.350503964306627], [15, 3.500696638797234], [16, 3.7746639439224374], [17, 4.0368449771334145], [18, 2.9002080229099527], [19, 2.7079782326069686], [20, 3.2998918421114922], [21, 3.878697838678395], [22, 3.5831226935955605], [23, 3.4372223505293875], [24, 3.42865353585338], [25, 3.6390748285840546], [26, 2.7503238446438916], [29, 4.12531550504561], [31, 3.133728951950703], [32, 4.071461086648935], [34, 3.865808321411683], [36, 3.841573578580747], [39, 3.6774218100991263], [41, 3.4234461479949223], [43, 2.751179325901085], [44, 3.2501575514270526], [45, 3.625478356645419], [46, 3.4998559415472585], [47, 4.270845250221253], [48, 3.3210286097581436], [50, 4.4622440514493045], [52, 3.533561041590749], [57, 3.2497850333057596], [58, 4.0954

[[1, 4.209949405075008], [2, 3.943193157960945], [3, 3.1594771395976], [5, 3.282278974645366], [6, 3.7631163348801424], [7, 3.4166242243061244], [9, 3.5003675736914643], [10, 3.1247477505611605], [11, 4.250031839854701], [16, 3.9999506689888364], [17, 3.9005231647816427], [19, 3.4169024736566382], [24, 3.333862465867033], [25, 3.388417729575534], [29, 4.454211694680843], [31, 4.082799123523979], [32, 4.138172535894542], [34, 3.8001311794457697], [36, 3.9452608096805637], [39, 3.9501626823935347], [44, 2.786077469673905], [47, 4.265719413189611], [48, 3.222149119404348], [50, 4.500225969633291], [58, 3.999080202569464], [60, 2.9360792652318444], [62, 4.092169557471896], [65, 1.9287436147881039], [69, 3.749460641844652], [70, 2.915163433641474], [73, 4.499908710324412], [74, 3.498757868626086], [79, 3.584043690004758], [88, 3.500044909771784], [95, 3.408841039701465], [104, 3.6588916970580376], [105, 3.2498284240288196], [107, 3.300651521846212], [110, 4.3368029838052164], [111, 3.874358

[[1, 4.031299602756477], [2, 3.834124819037503], [3, 2.5014169705583797], [5, 3.438656006786908], [6, 3.9469605101109955], [7, 3.7142059835322465], [10, 3.5792899810342553], [11, 4.074041408171371], [16, 3.7501437430197764], [17, 4.249945918241908], [19, 2.8692287526395583], [21, 3.4441561220932266], [22, 3.3189848591466924], [24, 3.636193166960827], [25, 3.714680025705974], [28, 4.198593353682848], [29, 4.2499305084215155], [31, 3.4617709341693366], [32, 4.031247104722633], [34, 3.986543919551011], [36, 4.569145101123943], [39, 3.4564792331547967], [41, 4.599847151720042], [43, 3.7995251120238938], [44, 2.28501139920974], [45, 3.5625872738730364], [47, 4.063656276825092], [48, 2.6432049940547624], [50, 4.505559993183987], [52, 3.77790434351759], [58, 4.2622900068755545], [62, 4.249932496651437], [73, 2.99880331069418], [74, 3.4997351219184], [86, 4.000667399120168], [94, 3.19947476312465], [95, 2.9379415540915117], [101, 4.100407085315157], [104, 3.4583530265594176], [105, 3.151171245

[[1, 4.05308309891091], [2, 3.499788423415564], [3, 2.9999122377639327], [5, 2.749685679013672], [6, 4.334176701338873], [7, 3.2519246740500254], [10, 3.4998436849362387], [11, 3.572214573228376], [14, 3.0000000000000004], [16, 4.138722979011711], [17, 3.8084344443859734], [19, 3.549559219868806], [20, 3.0018305885888754], [21, 3.2272646937531153], [22, 3.0], [23, 3.499688339890821], [25, 4.832552623495957], [29, 4.332437389679474], [31, 3.8121206532120375], [32, 4.073967727130577], [34, 3.83311700973058], [36, 4.0002417553413965], [39, 3.278003332511093], [44, 2.2481443686074067], [47, 4.3150362022651745], [48, 3.166101020319969], [50, 4.484496034193379], [60, 2.3559231969951497], [62, 3.4529355079123327], [65, 1.501839994020989], [69, 4.501622988875575], [70, 3.8747506939148653], [79, 3.9986577316960803], [88, 3.8756725534849057], [95, 3.7493416438430534], [104, 4.101711502245672], [105, 3.3557667168041228], [107, 3.8573421966265835], [110, 4.427272865766669], [111, 4.019263830265183

[[1, 4.2272763233104795], [2, 3.3750194900586172], [5, 2.666172416468144], [6, 4.218876239275157], [10, 3.8001866245354927], [11, 3.6254782465026127], [16, 4.029397513967876], [17, 4.4738777852247384], [19, 2.2497217703461723], [21, 3.6924980367542815], [22, 3.8124652308802887], [24, 3.49952025558923], [25, 4.269474953597002], [28, 4.749463063971167], [29, 4.400272694201682], [31, 3.5006143686957496], [32, 4.148444936817377], [34, 3.8530933661419366], [36, 4.208260518339975], [39, 4.078877920387529], [44, 2.213712072441544], [45, 2.9998077839869026], [47, 4.648274835489665], [48, 2.1666249062294884], [50, 4.738082109584953], [52, 3.5993055414584925], [58, 4.199792514859703], [60, 3.8754811903239235], [62, 3.30087694975874], [69, 3.78539801806378], [70, 3.436858481135857], [95, 3.4998994150599354], [101, 4.374655432584959], [104, 3.4278108681537165], [105, 3.166364937663158], [107, 3.5993779841049274], [110, 4.600020839336029], [111, 4.634547982333772], [112, 3.428383528624691], [135, 3

...

[[1, 4.144731077854322], [2, 3.4381388034044584], [3, 3.416811108433962], [5, 3.4371623694527287], [6, 3.401698759546587], [7, 3.600926391777882], [10, 3.9498799779372575], [19, 3.2994800496380874], [22, 3.415761542073888], [29, 3.9999577011709317], [31, 3.1258628749937944], [32, 4.077163541707183], [36, 4.6004349304221135], [44, 2.92300779241147], [47, 4.3000280811998675], [48, 2.9266472851948153], [50, 4.450305701555664], [58, 3.6652427934190075], [62, 3.500848610960093], [65, 2.496397191688989], [69, 4.110682704308162], [70, 4.12646178596241], [104, 3.857036536083376], [105, 4.123806985154954], [107, 3.2516696623863495], [110, 4.534956408089118], [111, 3.9999952183704752], [112, 3.376224348246314], [141, 2.899810900667066], [145, 3.856904661218869], [147, 4.249357779549024], [150, 4.118797656525642], [151, 3.8747055288682426], [153, 3.83346838352232], [158, 1.9995692717000098], [161, 3.7858595127703296], [163, 3.2260657145891516], [165, 4.269463982575371], [168, 2.888239475610593], 

[[1, 4.668321621283977], [2, 3.5152805512244316], [3, 3.400236722668268], [4, 3.750057829733867], [5, 3.9583789900110458], [6, 4.085749996373756], [7, 3.4615249285394847], [10, 3.5985162250580993], [11, 4.173205715911455], [14, 4.16669149397363], [16, 4.02561202429185], [17, 4.177491779539216], [18, 3.0002555829064677], [19, 2.5892823610381988], [20, 2.500065726711474], [21, 3.954557809862555], [22, 3.6000196291330417], [23, 2.642833259874737], [24, 3.0620916866151626], [25, 3.787919697226251], [28, 4.0003694397329905], [29, 4.00018228555407], [31, 3.611678112775669], [32, 4.011730819314875], [34, 4.0692379560648595], [36, 4.290367577479117], [39, 3.543387308833003], [41, 2.999799987868774], [44, 2.400077016681348], [45, 3.8334095440493052], [47, 4.155461681261337], [48, 3.1002153939012205], [50, 4.465394444283503], [52, 3.593642803110341], [57, 3.4281933750321154], [58, 3.7669883197739145], [60, 2.7500795665407964], [62, 3.964359865168809], [63, 2.625000844763887], [65, 1.666476077482

[[1, 4.0447178343514265], [2, 3.166098328802762], [3, 3.201714132694084], [5, 3.6691105944145432], [6, 4.749079362974478], [7, 3.0022182124126964], [11, 3.9088988133963536], [14, 3.600064753490734], [16, 4.2518534041255], [17, 4.374489582811555], [19, 2.9179300254919385], [21, 3.4265478995692598], [25, 3.6997214376395045], [29, 4.054055051736215], [31, 3.49965826922996], [32, 3.941893203967113], [34, 3.5003022942483812], [36, 4.025248336640714], [39, 3.7199145933766182], [44, 2.2469119363118857], [45, 2.799907814105601], [47, 4.128187239196188], [48, 3.0004769417167196], [50, 4.427875665250428], [52, 3.601479341960223], [58, 4.461574042427492], [62, 3.9449817254572985], [65, 1.9172539766400352], [69, 3.500840762801707], [73, 4.07069256258934], [95, 3.1992944098748946], [101, 3.9993621945643754], [104, 2.845710896702715], [105, 3.2207850455043374], [110, 4.000171304938273], [111, 4.2236265053254884], [112, 3.2527458087788954], [135, 2.399773119636282], [140, 3.5006404139224205], [141, 3

[[1, 4.0714052681005874], [2, 3.479637161372956], [3, 3.656661380713648], [5, 3.000395157391983], [6, 4.042978885510209], [7, 4.111595139793008], [10, 3.3752779437505787], [11, 3.619402313055176], [12, 3.400298038773761], [13, 3.7497534351637416], [14, 3.4168256292059858], [16, 4.10351038183588], [17, 4.106090304156185], [18, 2.999906282391577], [19, 2.5345064417962275], [20, 3.334352348715656], [21, 3.871925741466846], [22, 3.210705567953595], [23, 3.437440659440188], [24, 3.428792493797823], [25, 4.064481159204265], [28, 3.8333001650629273], [29, 3.929415717245988], [31, 3.423610555402304], [32, 4.021815760556691], [34, 3.6957792012108945], [36, 4.145288857749169], [39, 3.5256520835907055], [41, 4.665937039069682], [43, 3.799787305147138], [44, 2.8422299398469035], [45, 3.7629440827034144], [47, 4.08813499375315], [48, 3.273138370704993], [50, 4.472381269110208], [52, 3.93777604548674], [57, 3.3998086044641096], [58, 4.07507965171458], [60, 3.750006466295358], [61, 3.5000309346996774

,userId,movieId,rating
0,1,2,3.970709
1,11,527,4.568207
2,35,110,4.600021
3,50,1094,3.570565
4,59,1,4.668322
5,83,342,3.789501


In [1]:
realdata_rating = []
for userid in resultdf['userId']:
    realdata_rating.append(float(selected_user[selected_user['userId'] == userid]['rating']))

resultdata_rating = resultdf.rating.tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)
print("\nError Rate(Absolute) : ", error_rate_absol)
print("Error Rate(Squared):" , error_rate_squared)

NameError: name 'resultdf' is not defined